In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
import cv2 as cv

train=pd.read_csv('train.csv')
os.chdir('Train Images')
X=train['Image'].values
Xtrain=[]
for i in X:
    im = cv.imread(i)
    im = cv.resize(im,(96,96),interpolation=cv.INTER_LINEAR)
    Xtrain.append(new)
Xtrain=np.array(Xtrain)    
Xtest,Xtrain=Xtrain[:983],Xtrain[983:]
train[train['Class']=='Food']=0
train[train['Class']=='misc']=1
train[train['Class']=='Attire']=2
train[train['Class']=='Decorationandsignage']=3
ytrain=train['Class'].values
ytest,ytrain=ytrain[:983],ytrain[983:]
os.chdir('C:\Windows\System32\ML_PATH')

In [21]:
Xtrain.shape

(5000, 96, 96, 3)

In [22]:
Xtrain=Xtrain.astype('float32')
Xtest=Xtest.astype('float32')
ytrain=ytrain.astype('float32')
ytest=ytest.astype('float32')
m,s=Xtrain.mean(),Xtrain.std()
def standardize(data):
    mean,std=data.mean(),data.std()
    data=(data-mean)/std
    return data   
Xtrain=standardize(Xtrain)
Xtest=(Xtest-m)/s
Xtrain,Xval=Xtrain[:4000],Xtrain[4000:]
ytrain,yval=ytrain[:4000],ytrain[4000:]

In [23]:
from sklearn.utils import class_weight
a=class_weight.compute_class_weight('balanced',np.array([0.,1.,2.,3.]),ytrain)

In [24]:
base_model=tf.keras.applications.MobileNetV2(input_shape=[96,96,3],include_top=False,weights='imagenet')

In [25]:
base_model.trainable=False

In [26]:
avg=keras.layers.GlobalAveragePooling2D()
d1=keras.layers.Dense(1024,activation='elu',kernel_initializer='he_normal')
bn1=keras.layers.BatchNormalization()
dr1=keras.layers.Dropout(0.50)
d2=keras.layers.Dense(512,activation='elu',kernel_initializer='he_normal')
bn2=keras.layers.BatchNormalization()
dr2=keras.layers.Dropout(0.25)
d3=keras.layers.Dense(256,activation='elu',kernel_initializer='he_normal')
bn3=keras.layers.BatchNormalization()
dr3=keras.layers.Dropout(0.25)
d4=keras.layers.Dense(128,activation='elu',kernel_initializer='he_normal')
bn4=keras.layers.BatchNormalization()
output=keras.layers.Dense(4,activation='softmax')
model=tf.keras.Sequential([base_model,avg,output])
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=1e-3,beta_1=0.9,beta_2=0.999),metrics=['accuracy'])

In [27]:
n_epochs = 20
batch_size=32
def exp(epoch):
    return  0.005*0.1**(epoch/60)
lr=keras.callbacks.LearningRateScheduler(exp)
history = model.fit(Xtrain, ytrain,batch_size=batch_size, epochs=n_epochs, validation_data=[Xval,yval],class_weight=a)          

Train on 4000 samples, validate on 1000 samples
Epoch 1/20
4000/4000 [==============================] - 74s 19ms/sample - loss: 1.4437 - accuracy: 0.3808 - val_loss: 1.6794 - val_accuracy: 0.3990
Epoch 2/20
4000/4000 [==============================] - 74s 18ms/sample - loss: 1.2542 - accuracy: 0.4485 - val_loss: 1.6335 - val_accuracy: 0.3890
Epoch 3/20
4000/4000 [==============================] - 74s 18ms/sample - loss: 1.1843 - accuracy: 0.4855 - val_loss: 1.5117 - val_accuracy: 0.3700
Epoch 4/20
4000/4000 [==============================] - 74s 18ms/sample - loss: 1.1425 - accuracy: 0.4960 - val_loss: 1.5234 - val_accuracy: 0.3800
Epoch 5/20
4000/4000 [==============================] - 73s 18ms/sample - loss: 1.1088 - accuracy: 0.5188 - val_loss: 1.4899 - val_accuracy: 0.3830
Epoch 6/20
4000/4000 [==============================] - 75s 19ms/sample - loss: 1.0842 - accuracy: 0.5360 - val_loss: 1.5886 - val_accuracy: 0.3870
Epoch 7/20
4000/4000 [==============================] - 76s 19ms

In [9]:
history = model.fit_generator(datagen.flow(Xtrain, ytrain,batch_size=batch_size), epochs=40,initial_epoch=20,validation_data=[Xval,yval],class_weight=a) 

  ...
    to  
  ['...']
Train for 125 steps, validate on 1000 samples
Epoch 21/40
125/125 [==============================] - 118s 943ms/step - loss: 0.7678 - accuracy: 0.6902 - val_loss: 0.8867 - val_accuracy: 0.6460
Epoch 22/40
125/125 [==============================] - 118s 945ms/step - loss: 0.7322 - accuracy: 0.7020 - val_loss: 0.9576 - val_accuracy: 0.6200
Epoch 23/40
125/125 [==============================] - 119s 951ms/step - loss: 0.7391 - accuracy: 0.7023 - val_loss: 1.0189 - val_accuracy: 0.6120
Epoch 24/40
125/125 [==============================] - 117s 939ms/step - loss: 0.7112 - accuracy: 0.7125 - val_loss: 0.8534 - val_accuracy: 0.6390
Epoch 25/40
125/125 [==============================] - 118s 943ms/step - loss: 0.6909 - accuracy: 0.7295 - val_loss: 0.8977 - val_accuracy: 0.6520
Epoch 26/40
125/125 [==============================] - 118s 941ms/step - loss: 0.6897 - accuracy: 0.7265 - val_loss: 1.0482 - val_accuracy: 0.6210
Epoch 27/40
125/125 [==========================

In [ ]:
model.evaluate(Xtrain,ytrain)

2112/4000 [==============>...............] - ETA: 8s - loss: 1.0177 - accuracy: 0.6411